In [22]:
import sklearn
import nltk
import re 
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn import preprocessing
from sklearn import metrics
from sklearn.feature_selection import SelectKBest
from sklearn.feature_extraction.text import TfidfVectorizer #as vectorizer
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import chi2
from sklearn.metrics import precision_score, recall_score,f1_score
from sklearn.linear_model import LogisticRegression

##########################################################################3
##prepare dataset
from os import getcwd, chdir
import pandas as pd
import pickle as pk
fpath = getcwd()
print (fpath)
# Change your path here
chdir(fpath) 



C:\Users\user\OneDrive - National University of Singapore\EBA5004\project


In [23]:
with open('AMAZON_FASHION.npy', 'rb') as f:
    a = np.load(f, allow_pickle=True)
df_data=pd.DataFrame(a,columns=['asin', 'overall', 'reviewText', 'reviewTime', 'reviewerID',
       'reviewerName', 'summary', 'unixReviewTime', 'verified', 'vote',
       'style', 'image'
])
subset_data=df_data

In [24]:
subset_data

,asin,overall,reviewText,reviewTime,reviewerID,reviewerName,summary,unixReviewTime,verified,vote,style,image
0,7106116521,5,Exactly what I needed.,"10 20, 2014",A1D4G1SNUZWQOT,Tracy,perfect replacements!!,1.41376e+09,1,NaN,NaN,NaN
1,7106116521,2,"I agree with the other review, the opening is ...","09 28, 2014",A3DDWDH9PX2YX2,Sonja Lau,"I agree with the other review, the opening is ...",1.41186e+09,1,3,NaN,NaN
2,7106116521,4,Love these... I am going to order another pack...,"08 25, 2014",A2MWC41EW7XL15,Kathleen,My New 'Friends' !!,1.40892e+09,0,NaN,NaN,NaN
3,7106116521,2,too tiny an opening,"08 24, 2014",A2UH2QQ275NV45,Jodi Stoner,Two Stars,1.40884e+09,1,NaN,NaN,NaN
4,7106116521,3,Okay,"07 27, 2014",A89F3LQADZBS5,Alexander D.,Three Stars,1.40642e+09,0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
883631,B01HJHTH5U,5,I absolutely love this dress!! It's sexy and ...,"02 21, 2017",A1ZSB2Q144UTEY,Amazon Customer,I absolutely love this dress,1.48764e+09,1,NaN,NaN,NaN
883632,B01HJHTH5U,5,I'm 5'6 175lbs. I'm on the tall side. I wear a...,"11 25, 2016",A2CCDV0J5VB6F2,Amazon Customer,I wear a large and ordered a large and it stil...,1.48003e+09,1,2,NaN,NaN
883633,B01HJHTH5U,3,Too big in the chest area!,"11 10, 2016",A3O90PACS7B61K,Fabfifty,Three Stars,1.47874e+09,1,NaN,NaN,NaN
883634,B01HJHF97K,3,"Too clear in the back, needs lining","11 10, 2016",A2HO94I89U3LNH,Mgomez,Three Stars,1.47874e+09,1,NaN,NaN,NaN


In [25]:
subset_data.loc[subset_data['overall'] <= 3, 'equal_or_lower_than_3?'] = -1
subset_data.loc[subset_data['overall'] > 3, 'equal_or_lower_than_3?'] = 1 

print (subset_data)

              asin overall                                         reviewText  \
0       7106116521       5                             Exactly what I needed.   
1       7106116521       2  I agree with the other review, the opening is ...   
2       7106116521       4  Love these... I am going to order another pack...   
3       7106116521       2                                too tiny an opening   
4       7106116521       3                                               Okay   
...            ...     ...                                                ...   
883631  B01HJHTH5U       5  I absolutely love this dress!!  It's sexy and ...   
883632  B01HJHTH5U       5  I'm 5'6 175lbs. I'm on the tall side. I wear a...   
883633  B01HJHTH5U       3                         Too big in the chest area!   
883634  B01HJHF97K       3                Too clear in the back, needs lining   
883635  B01HJG5NMW       5  Ordered and was slightly small. Worked with th...   

         reviewTime      re

In [26]:
X=subset_data['reviewText'].apply(str).tolist()
Y=subset_data['equal_or_lower_than_3?'].tolist()
len(Y)

883636

In [27]:
X_train=X[:int(len(Y)*0.8)]
X_test=X[int(len(Y)*0.8):]
Y_train=Y[:int(len(Y)*0.8)]
Y_test=Y[int(len(Y)*0.8):]

In [28]:
X_train

['Exactly what I needed.',
 "I agree with the other review, the opening is too small.  I almost bent the hook on some very expensive earrings trying to get these up higher than just the end so they're not seen.  Would not buy again but for the price, not sending back.",
 "Love these... I am going to order another pack to keep in work; someone (including myself) is always losing the back to an earring.  I don't understand why all fish hook earrings don't have them.  Just wish that they were a tiny bit longer.  :)",
 'too tiny an opening',
 'Okay',
 'Exactly what I wanted.',
 'These little plastic backs work great.  No more loosing "hook" earrings!  Wish I had ordered these sooner before I had lost some of my favorite earrings!',
 'mother - in - law wanted it as a present for her sister. she liked it and said it would work.',
 "Item is of good quality. Looks great, too. But it does not fit 100's! But it can be stretched to fit them if you carefully push bottom of case with your fingers. 

In [29]:
len(X_test)

176728

In [30]:
#Vectorizer the sentences using Tfidf vale
#Make sure test data should be transformed using vectorizer learned from trainning data 
vectorizer = TfidfVectorizer(ngram_range=(1, 2), min_df=1)#unigram and bigram
train_vectors = vectorizer.fit_transform(X_train)
test_vectors = vectorizer.transform(X_test)

# same feature set

test_vectors.shape

(176728, 1715877)

In [31]:
train_vectors.shape

(706908, 1715877)

In [114]:
#########################################################
#Apply NB model
clf_NB = MultinomialNB().fit(train_vectors, Y_train)
predNB = clf_NB.predict(test_vectors)
pred = list(predNB)



'''
errors=[]
for a,b, index in pred,Y_test:
    if a==b:
        continue
    else:
        errors.append(test_set[index])
print (errors)    
'''

print(metrics.confusion_matrix(Y_test, pred))
print(metrics.classification_report(Y_test, pred))

[[ 34174  28924]
 [  1641 111989]]
              precision    recall  f1-score   support

        -1.0       0.95      0.54      0.69     63098
         1.0       0.79      0.99      0.88    113630

    accuracy                           0.83    176728
   macro avg       0.87      0.76      0.79    176728
weighted avg       0.85      0.83      0.81    176728



In [32]:
#########################################################
# MaxEnt = LogisticRegression
clf_ME = LogisticRegression(random_state=0, solver='lbfgs').fit(train_vectors, Y_train)
predME = clf_ME.predict(test_vectors)
pred = list(predME)
print(metrics.confusion_matrix(Y_test, pred))
print(metrics.classification_report(Y_test, pred))

C:\Users\user\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


[[ 53542   9556]
 [  6697 106933]]
              precision    recall  f1-score   support

        -1.0       0.89      0.85      0.87     63098
         1.0       0.92      0.94      0.93    113630

    accuracy                           0.91    176728
   macro avg       0.90      0.89      0.90    176728
weighted avg       0.91      0.91      0.91    176728



In [33]:
filename = 'logreg_model.sav'
pk.dump(clf_ME, open(filename, 'wb'))
with open('vectorizer.pk', 'wb') as fin:
    pk.dump(vectorizer, fin)

In [34]:
vectorizer2 = pk.load(open('vectorizer.pk', 'rb'))
loaded_model = pickle.load(open(filename, 'rb'))
predME = loaded_model.predict(test_vectors)
pred = list(predME)
print(metrics.confusion_matrix(Y_test, pred))
print(metrics.classification_report(Y_test, pred))

[[ 53542   9556]
 [  6697 106933]]
              precision    recall  f1-score   support

        -1.0       0.89      0.85      0.87     63098
         1.0       0.92      0.94      0.93    113630

    accuracy                           0.91    176728
   macro avg       0.90      0.89      0.90    176728
weighted avg       0.91      0.91      0.91    176728



In [35]:
test_vectors2 = vectorizer2.transform(["this shirt is not pretty"])
predME = loaded_model.predict(test_vectors2)
pred = list(predME)
pred

[-1.0]

In [116]:
#####KNN Classifier
def train_knn(X, y, k, weight):
    """
    Create and train the k-nearest neighbor.
    """
    knn = KNeighborsClassifier(n_neighbors = k, weights = weight, metric = 'cosine', algorithm = 'brute')
    knn.fit(X, y)
    return knn


kn = train_knn(train_vectors, Y_train, 3, 'distance')# distance weights - by inverse of distance
predKN = kn.predict(test_vectors)
pred = list(predKN)
print(metrics.confusion_matrix(Y_test, pred))
print(metrics.classification_report(Y_test, pred))

[[ 45434  17664]
 [ 10995 102635]]
              precision    recall  f1-score   support

        -1.0       0.81      0.72      0.76     63098
         1.0       0.85      0.90      0.88    113630

    accuracy                           0.84    176728
   macro avg       0.83      0.81      0.82    176728
weighted avg       0.84      0.84      0.84    176728



In [117]:
#Apply SVM model
'''
Not all the feature points are needed to define the support vector.
In a way only the most informative feature points help to define the support vector which makes the SVM ideal for high-dimensional classification 
and sparse matrics common in NLP. 
The SVM can also work on feature points that are not linearly separable through kernel transformation of the feature points.
0.85 ACC
0.85 F1
'''

from sklearn import svm
from sklearn.svm import LinearSVC

model_svm = LinearSVC(C=100)
clr_svm = model_svm.fit(train_vectors, Y_train)


predicted = clr_svm.predict(test_vectors)
 
print(metrics.confusion_matrix(Y_test, predicted))
print(np.mean(predicted == Y_test) )
print(metrics.classification_report(Y_test, predicted))

C:\Users\user\anaconda3\lib\site-packages\sklearn\svm\_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "


[[ 50151  12947]
 [ 11289 102341]]
0.8628627042687067
              precision    recall  f1-score   support

        -1.0       0.82      0.79      0.81     63098
         1.0       0.89      0.90      0.89    113630

    accuracy                           0.86    176728
   macro avg       0.85      0.85      0.85    176728
weighted avg       0.86      0.86      0.86    176728



In [122]:
print(train_vectors)

  (0, 1649230)	0.5849470830192129
  (0, 493533)	0.46348302998369445
  (0, 958650)	0.4389468387803993
  (0, 1648362)	0.3102000709766462
  (0, 493166)	0.39258372314096995
  (1, 1262837)	0.13652022915556866
  (1, 985887)	0.16969047125028697
  (1, 1138200)	0.13878650433698658
  (1, 1462343)	0.06503450582147204
  (1, 568424)	0.059504964460325334
  (1, 257551)	0.0980048122072092
  (1, 62082)	0.12965521522167392
  (1, 262958)	0.09619367395398801
  (1, 982235)	0.10142658663857318
  (1, 1691421)	0.08968223011003797
  (1, 1259621)	0.1937734191020609
  (1, 985876)	0.14917367381253127
  (1, 1178471)	0.11460997850692153
  (1, 1482723)	0.08340972471996141
  (1, 1341074)	0.10104958965843636
  (1, 475883)	0.16402910230230255
  (1, 1455414)	0.10516197995525187
  (1, 784991)	0.0984719959851576
  (1, 1442052)	0.1372599971474804
  (1, 685041)	0.1316634327204454
  :	:
  (706906, 997989)	0.07985617508355757
  (706906, 1470583)	0.04965198378094799
  (706906, 561182)	0.03423582896393911
  (706906, 1481241)	0.

In [158]:
###############################################
##Select K Best features

ch21 = SelectKBest(chi2, k=15000)
# Transform your training and testing datasets accordingly
train_Kbest = ch21.fit_transform(train_vectors, Y_train)
test_Kbest = ch21.transform(test_vectors)
train_Kbest.shape

(706908, 15000)

In [159]:
#Re-Train ME model
clf_ME = LogisticRegression(random_state=0, solver='lbfgs').fit(train_Kbest, Y_train)
predME = clf_ME.predict(test_Kbest)
pred = list(predME)
print(metrics.confusion_matrix(Y_test, pred))
print(metrics.classification_report(Y_test, pred))

C:\Users\user\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


[[ 53014  10084]
 [  6960 106670]]
              precision    recall  f1-score   support

        -1.0       0.88      0.84      0.86     63098
         1.0       0.91      0.94      0.93    113630

    accuracy                           0.90    176728
   macro avg       0.90      0.89      0.89    176728
weighted avg       0.90      0.90      0.90    176728



In [125]:
train_vectors.shape

(706908, 1715877)